In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from data import DataGenerator
from model import espcn, psnr
from testing import benchmark
import random

weights_filename = 'r3bs10epochs100weights.h5' # filepath of weights
r = 3 # Upscale Factor

# Compile model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# Peak Signal-to-Noise Ratio
def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)
model = espcn(r)
model.compile(optimizer=opt, loss='mse', metrics=[PSNR])
# Load weights
filepath = 'model/weights/' + weights_filename
model.load_weights(filepath)
model.summary()
# Initialize generator
testing_generator = DataGenerator('LRbicx' + str(r), batch_size = 1, dictionary = "test")

In [ ]:
# Generate pair and super-resolve LR image
lr, hr = testing_generator.__getitem__(random.randint(0,729))
sr = model.predict(lr)
# Show LR, SR, and HR images side-by-side
fig = plt.figure()
fig.set_size_inches(28, 28)
ax1 = fig.add_subplot(1,3,1)
ax1.set_title('Low Resolution (LR): ' + str(lr[0].shape[0]) + ' x ' + str(lr[0].shape[1]) + ' pixels')
ax1.imshow(lr[0])
ax2 = fig.add_subplot(1,3,2)
ax2.set_title('Super Resolution (SR): ' + str(sr[0].shape[0]) + ' x ' + str(sr[0].shape[1]) + ' pixels')
ax2.imshow(sr[0])
ax3 = fig.add_subplot(1,3,3)
ax3.set_title('High Resolution (HR): ' + str(hr[0].shape[0]) + ' x ' + str(hr[0].shape[1]) + ' pixels')
ax3.imshow(hr[0])
plt.show()
# Calculate PSNR
print("PSNR: " + str(psnr(sr[0], hr[0])))

In [ ]:
# Calculate Average PSNR
benchmark(weights_filename, r, 'True')